In [8]:
!pip install jupyter-dash

     |████████████████████████████████| 45 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 61 kB 420 kB/s eta 0:00:011
     |████████████████████████████████| 126 kB 5.1 MB/s eta 0:00:01
  Created wheel for ansi2html: filename=ansi2html-1.5.2-py3-none-any.whl size=13860 sha256=0fc4b9a8e2641426c6784fb8971cdb91c819387235bfd80c6f861c875ff6a540
  Stored in directory: /home/jovyan/data/.cache/pip/wheels/62/fa/1e/1ad9fb5c2225613adfcf012dd25f313f3a065f2131cdff0a99
Successfully built ansi2html


In [6]:
import plotly.graph_objects as go # or plotly.express as px
fig = go.Figure() # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )

import dash
import dash_core_components as dcc
import dash_html_components as html

In [7]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(debug=True, use_reloader=False)  # Turn off reloader

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [2]:
import plotly.graph_objects as go
import numpy as np

x = np.arange(10)

fig = go.Figure(data=go.Scatter(x=x, y=x**2))
fig.show()

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider

Loading BokehJS ...

In [2]:

from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider
from bokeh.themes import Theme
from bokeh.io import show, output_notebook

from bokeh.models import ColumnDataSource, DataRange1d, Select, RadioButtonGroup

# output_notebook()

import pandas as pd
from bokeh.io import curdoc

In [3]:
plot = figure(x_axis_type="datetime",
              plot_width=800,)
plot.title.text = "Covid Deaths"

In [4]:
df = pd.read_csv('dat/covid-deaths.tsv',
                 delimiter='\t')

In [5]:
state = 'Massachusetts'
states = df.columns[1:]
state_select = Select(value=state,
                      title='State',
                      options=sorted(states))
mode_select = Select(value='Cumulative',
                     title='Mode',
                     options=['Cumulative',
                              'Incremental',
                              'Percent Change'])
agg_select = Select(value='Cumulative',
                    title='Aggregation',
                    options=['Daily',
                             'Weekly',
                             '7-Day Ave'])
style_select = Select(value='Line',
                        title='Style',
                        options=['Line',
                                 'Bar'])

In [6]:
dates = pd.to_datetime(df.date, format='%Y%m%d')
source = ColumnDataSource(data=dict(x=dates,
                                    y=df[state]))

line = plot.line('x', 'y',
                 source=source,
                 line_width=3,
                 line_alpha=0.6)
vbar = plot.vbar(x='x', top='y', width=0.2,
                 source=source)
vbar.visible = False

In [7]:
# fixed attributes
plot_title = 'COVID Deceased: ' 
plot.xaxis.axis_label = None
plot.yaxis.axis_label = "Count"
plot.axis.axis_label_text_font_style = "bold"
plot.x_range = DataRange1d(range_padding=0.0)
plot.grid.grid_line_alpha = 0.3

In [8]:
def update_plot(attrname, old, new):
    state = state_select.value
    plot.title.text = f"{plot_title}{state}"
    source.data = dict(x=dates, y=df[state])
    update_style(None, None, style_select.value)
    update_mode(None, None, mode_select.value)
    update_agg(None, None, agg_select.value)

def update_style(attrname, old, new):
    style = new.lower()
    if style == 'line':
        line.visible = True
        vbar.visible = False
    if style == 'bar':
        line.visible = False
        vbar.visible = True

def update_mode(attrname, old, new):
    agg = new.lower()
    state = state_select.value
    if agg == 'cumulative':
        # plot raw cumulative data
        source.data = dict(x=dates, y=df[state])
    if agg == 'incremental':
        # compute incremental
        diffs = df[state].diff()
        diffs[diffs < 0] = 0
        source.data = dict(x=dates,
                           y=diffs)
    if agg == 'percent change':
        # compute percent change
        pct = df[state].diff().pct_change()
        pct = pct.fillna(0)
        source.data = dict(x=dates,
                           y=pct)

def update_agg(attrname, old, new):
    # create new dataframe from current x,y data
    # then perform date agg
    df = pd.DataFrame(source.data)
    df = df.set_index('x')

    if new == 'weekly':
        df = df.groupby(pd.TimeGrouper('W')).sum().dropna()
    if new == 'daily':
        df = df.groupby(pd.TimeGrouper('D')).sum().dropna()

    source.data = dict(x=df.index, y=df.y)

    print('Update Agg not implemented')

def update_data(attrname, old, new):
    global df, plot_title
    selected = data_select.labels[new]
    if selected == 'Confirmed':
        df = pd.read_csv('dat/covid-confirmed.tsv',
                         delimiter='\t')
        plot_title = 'COVID Confirmed: ' 
    else:
        df = pd.read_csv('dat/covid-deaths.tsv',
                         delimiter='\t')
        plot_title = 'COVID Deceased: ' 
    update_plot('', '', '')

In [9]:
data_select = RadioButtonGroup(labels=['Confirmed', 'Deceased'], active=1)

state_select.on_change('value', update_plot)
style_select.on_change('value', update_style)
mode_select.on_change('value', update_mode)
data_select.on_change('active', update_data)


In [10]:
controls = column(data_select,
                  state_select,
                  mode_select,
                  style_select,
                  agg_select)
curdoc().add_root(row(plot, controls))

In [11]:
show(row(plot, controls))

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

